In [2]:
import pandas as pd
import numpy as np


In [3]:
r_cols = ['user_id', 'prd_id','title', 'rating']
ratings = pd.read_csv('GAME1000.csv', sep=',', names=r_cols, usecols=range(4), encoding="ISO-8859-1",skiprows=1)
#ratings

In [4]:
ratings =ratings.drop_duplicates()
#ratings

In [5]:
prdRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating',fill_value=0)
prdRatings.head()


title,AC,FF15,FF7,GOW,GTA4,GTA5,LASTOFUS,MGS,MHW,RDR2
user_id,,,,,,,,,,
Abba,1,1,1,1,1,1,1,0,1,1
Adaline,1,0,1,1,1,1,1,0,1,1
Allie,1,1,1,1,1,1,1,1,1,1
Amaleta,1,1,1,1,1,1,1,1,1,1
Arlan,1,1,1,1,1,0,0,1,1,1


In [6]:
iRecurrence = 1
prdStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
popularPrdTmp = prdStats['rating']['size'] >= iRecurrence
popularPrd = prdStats[popularPrdTmp].sort_values([('rating', 'mean')], ascending=False)


In [7]:
def recommendItem(sItemNm, dfPivotRating,dfPopularItems):
    
    inferenceRatings = dfPivotRating[sItemNm] 
    inferenceRatings.head()

    similarItems = dfPivotRating.corrwith(inferenceRatings,axis=0)
    similarItems = similarItems.dropna()
    similarItems = similarItems.sort_values(ascending=False)
    
    df = dfPopularItems.join(pd.DataFrame(similarItems, columns=['similarity']))
    df = df.sort_values(['similarity'], ascending=False)
    return df
   
    
#dfTmp = recommendItem("FF7",prdRatings,popularPrd)
#print(dfTmp.head())
  


In [8]:
cols_nm = ['title']
for col in prdRatings.columns:
    cols_nm.append(col)

dfResult = pd.DataFrame(columns=cols_nm)

for col in prdRatings.columns: 
    dfTmp = recommendItem(col,prdRatings,popularPrd)
    dTmp = {}
    dTmp['title'] = col
    for idx, row in dfTmp.iterrows():         
        dTmp[idx] = row['similarity']
    dfResult = dfResult.append(dTmp , ignore_index=True)
    
#dTmp = {}
#dTmp['MGS'] = "gen"
#dTmp ['FF7'] = "mah"
#dfResult = dfResult.append(dTmp , ignore_index=True)


#dfResult

/home/yockgen/.local/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [9]:
lstCsv = []
strHeader = ""
for col in dfResult.columns:
    strHeader +=  col + "," 
lstCsv.append(strHeader[:-1] + "\n")  

for row in dfResult.iterrows(): 
    strRow = ""
    for col in row[1]:
        strRow += str(col) + ","  
    lstCsv.append(strRow[:-1] + "\n")  

#print (str(lstCsv))

In [10]:
f = open('temp.csv', 'w')
with f:
    for row in lstCsv:
       f.writelines(row)

f.close() 

In [11]:
f = open('temp.csv', 'r')
with f:
    for row in f:             
        print(row)
f.close() 

title,AC,FF15,FF7,GOW,GTA4,GTA5,LASTOFUS,MGS,MHW,RDR2

AC,0.9999999999999998,0.3831232818676386,-0.12309149097933259,-0.012815774135146653,-0.04754330548804468,-0.1489923873929707,0.0820609939862219,0.02837950236056587,0.23869648112406272,-0.1363636363636362

FF15,0.3831232818676386,1.0,-0.13449109746432153,-0.03900739260141337,-0.0751366573213385,-0.16279069767441898,-0.13449109746432153,0.0033222591362126012,-0.08235864093362708,0.028379502360565807

FF7,-0.1230914909793326,-0.13449109746432153,1.0,0.01735264020984513,-0.01609348498617553,0.05763904177042361,-0.11111111111111113,-0.13449109746432156,-0.0680413817439772,0.08206099398622192

GOW,-0.012815774135146655,-0.03900739260141337,0.017352640209845133,1.0,0.002513380092188777,0.11102104048094577,0.017352640209845144,-0.03900739260141339,-0.09563650695950078,-0.17301295082447832

GTA4,-0.04754330548804469,-0.0751366573213385,-0.016093484986175525,0.0025133800921887774,0.9999999999999999,0.0640053006811403,-0.17702833484793143,0.0

In [12]:
dfResult

,title,AC,FF15,FF7,GOW,GTA4,GTA5,LASTOFUS,MGS,MHW,RDR2
0,AC,1.000000,0.383123,-0.123091,-0.012816,-0.047543,-0.148992,0.082061,0.028380,0.238696,-0.136364
1,FF15,0.383123,1.000000,-0.134491,-0.039007,-0.075137,-0.162791,-0.134491,0.003322,-0.082359,0.028380
2,FF7,-0.123091,-0.134491,1.000000,0.017353,-0.016093,0.057639,-0.111111,-0.134491,-0.068041,0.082061
3,GOW,-0.012816,-0.039007,0.017353,1.000000,0.002513,0.111021,0.017353,-0.039007,-0.095637,-0.173013
4,GTA4,-0.047543,-0.075137,-0.016093,0.002513,1.000000,0.064005,-0.177028,0.064005,0.137973,-0.047543
5,GTA5,-0.148992,-0.162791,0.057639,0.111021,0.064005,1.000000,0.057639,0.169435,-0.082359,0.205751
6,LASTOFUS,0.082061,-0.134491,-0.111111,0.017353,-0.177028,0.057639,1.000000,-0.134491,0.272166,-0.123091
7,MGS,0.028380,0.003322,-0.134491,-0.039007,0.064005,0.169435,-0.134491,1.000000,-0.082359,0.028380
8,MHW,0.238696,-0.082359,-0.068041,-0.095637,0.137973,-0.082359,0.272166,-0.082359,1.000000,-0.075378
9,RDR2,-0.136364,0.028380,0.082061,-0.173013,-0.047543,0.205751,-0.123091,0.028380,-0.075378,1.000000
